<a href="https://colab.research.google.com/github/cij1012/3-min-pytorch/blob/main/5.CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

In [ ]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

In [ ]:
Epochs     = 40
Batch_size = 64

In [ ]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./.data',
                   train=True,
                   download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=Batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./.data',
                   train=False, 
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=Batch_size, shuffle=True)

In [ ]:
conv1 = nn.Conv2d(1, 10, kernel_size=5)
print(conv1)

Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))


In [ ]:
class Net(nn.Module):
  def __init__(self):
    super(Net,self).__init__()
    self.conv1 = nn.Conv2d(1, 10, kernel_size=5) # 커널사이즈 5*5, 스트라이드 1*1 패딩 x?
    self.conv2 = nn.Conv2d(10, 20 ,kernel_size=5)
    self.conv2_drop = nn.Dropout2d()
    self.fc1 = nn.Linear(320, 50)
    self.fc2 = nn.Linear(50,10)

  def forward(self, x):
    x = F.relu(F.max_pool2d(self.conv1(x),2))
    x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)),2))
    x = x.view(-1,320)
    x = F.relu(self.fc1(x))
    x = F.dropout(x, training=self.training)
    x = self.fc2(x)
    return x

In [ ]:
model = Net().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [ ]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % 200 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [ ]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)

            # 배치 오차를 합산
            test_loss += F.cross_entropy(output, target,
                                         reduction='sum').item()

            # 가장 높은 값을 가진 인덱스가 바로 예측값
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [ ]:
for epoch in range(1, Epochs + 1):
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(
          epoch, test_loss, test_accuracy))

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.322672
Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.107653
Train Epoch: 1 [25600/60000 (43%)]	Loss: 1.247984
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.591630
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.339223
[1] Test Loss: 0.2038, Accuracy: 94.05%
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.522009
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.334599
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.351840
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.349038
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.423551
[2] Test Loss: 0.1254, Accuracy: 95.91%
Train Epoch: 3 [0/60000 (0%)]	Loss: 0.450545
Train Epoch: 3 [12800/60000 (21%)]	Loss: 0.307412
Train Epoch: 3 [25600/60000 (43%)]	Loss: 0.358901
Train Epoch: 3 [38400/60000 (64%)]	Loss: 0.205298
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.365749
[3] Test Loss: 0.0981, Accuracy: 97.02%
Train Epoch: 4 [0/60000 (0%)]	Loss: 0.341764
Train Epoch: 4 [12800/60000 (21%)]	Loss: 0.259584
Train Epoch: 4 [25600/60000 (43%)]	Loss: 0.160615
